In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import numpy as np
import tensorflow as tf

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [3]:
import json

with open('train-test.json') as fopen:
    dataset = json.load(fopen)
    
with open('dictionary.json') as fopen:
    dictionary = json.load(fopen)

In [4]:
train_X = dataset['train_X']
train_Y = dataset['train_Y']
test_X = dataset['test_X']
test_Y = dataset['test_Y']

In [5]:
dictionary.keys()

dict_keys(['from', 'to'])

In [6]:
dictionary_from = dictionary['from']['dictionary']
rev_dictionary_from = dictionary['from']['rev_dictionary']

dictionary_to = dictionary['to']['dictionary']
rev_dictionary_to = dictionary['to']['rev_dictionary']

In [7]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [8]:
for i in range(len(train_X)):
    train_X[i] += ' EOS'
    
train_X[0]

'Rachel Pike : The science behind a climate headline EOS'

In [9]:
for i in range(len(test_X)):
    test_X[i] += ' EOS'
    
test_X[0]

'How can I speak in <NUM> minutes about the bonds of women over three generations , about how the astonishing strength of those bonds took hold in the life of a four - year - old girl huddled with her young sister , her mother and her grandmother for five days and nights in a small boat in the China Sea more than <NUM> years ago , bonds that took hold in the life of that small girl and never let go - - that small girl now living in San Francisco and speaking to you today ? EOS'

In [10]:
def pad_second_dim(x, desired_size):
    padding = tf.tile([[[0.0]]], tf.stack([tf.shape(x)[0], desired_size - tf.shape(x)[1], tf.shape(x)[2]], 0))
    return tf.concat([x, padding], 1)

def layer_normalization(inputs, block_name, reuse, epsilon=1e-8):
    with tf.variable_scope(block_name, reuse = reuse):
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))

        params_shape = inputs.get_shape()[-1:]
        gamma = tf.get_variable('gamma', params_shape, tf.float32, tf.ones_initializer())
        beta = tf.get_variable('beta', params_shape, tf.float32, tf.zeros_initializer())

        outputs = gamma * normalized + beta
        return outputs

def conv1d(input_, output_channels, block_name, reuse, dilation = 1, filter_width = 1, causal = False):
    with tf.variable_scope(block_name, reuse = reuse):
        w = tf.get_variable('w',  [1, filter_width, int(input_.get_shape()[-1]), output_channels],
                            tf.float32, tf.initializers.random_normal(stddev = 0.02))
        b = tf.get_variable('b',  [output_channels],
                            tf.float32, tf.zeros_initializer())
        if causal:
            padding = [[0, 0], [(filter_width - 1) * dilation, 0], [0, 0]]
            padded = tf.pad(input_, padding)
            input_expanded = tf.expand_dims(padded, dim = 1)
            out = tf.nn.atrous_conv2d(input_expanded, w, rate = dilation, padding = 'VALID') + b
        else:
            input_expanded = tf.expand_dims(input_, dim = 1)
            out = tf.nn.atrous_conv2d(input_expanded, w, rate = dilation, padding = 'SAME') + b
        return tf.squeeze(out, [1])

def bytenet_residual_block(input_, dilation, layer_no, 
                            residual_channels, filter_width, block_type,
                            causal = True, reuse = False):
    block_name = "bytenet_{}_layer_{}_{}".format(block_type, layer_no, dilation)
    print(block_name)
    with tf.variable_scope(block_name, reuse = reuse):
        relu1 = tf.nn.relu(layer_normalization(input_, block_name + '_0', reuse))
        conv1 = conv1d(relu1, residual_channels, block_name + '_0', reuse)
        relu2 = tf.nn.relu(layer_normalization(conv1, block_name + '_1', reuse))
        dilated_conv = conv1d(relu2, residual_channels,
                              block_name + '_1', reuse,
                              dilation, filter_width,
                              causal = causal)
        print(dilated_conv)
        relu3 = tf.nn.relu(layer_normalization(dilated_conv, block_name + '_2', reuse))
        conv2 = conv1d(relu3, 2 * residual_channels, block_name + '_2', reuse)
        return input_ + conv2
    
class ByteNet:
    def __init__(self, from_vocab_size, to_vocab_size, channels, encoder_dilations,
                decoder_dilations, encoder_filter_width, decoder_filter_width,
                learning_rate = 1e-3, beta1=0.5):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype = tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype = tf.int32)
        batch_size = tf.shape(self.X)[0]
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        target_1 = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        embedding_channels = 2 * channels
        max_seq = tf.maximum(tf.reduce_max(self.Y_seq_len), tf.reduce_max(self.X_seq_len))
        w_source_embedding = tf.Variable(tf.random_normal([from_vocab_size, 
                                                           embedding_channels], stddev = 0.02))
        w_target_embedding = tf.Variable(tf.random_normal([to_vocab_size, 
                                                           embedding_channels], stddev = 0.02))
        
        def forward(x, y, reuse = False):
            source_embedding = tf.nn.embedding_lookup(w_source_embedding, x)
            target_1_embedding = tf.nn.embedding_lookup(w_target_embedding, y)
        
        
            curr_input = source_embedding
            for layer_no, dilation in enumerate(encoder_dilations):
                curr_input = bytenet_residual_block(curr_input, dilation, 
                                                    layer_no, channels, 
                                                    encoder_filter_width,
                                                    'encoder',
                                                    causal = False, reuse = reuse)
            encoder_output = curr_input
            combined_embedding = target_1_embedding + encoder_output
            curr_input = combined_embedding
            for layer_no, dilation in enumerate(decoder_dilations):
                curr_input = bytenet_residual_block(curr_input, dilation, 
                                                    layer_no, channels, 
                                                    encoder_filter_width, 
                                                    'decoder',
                                                    causal = False, reuse = reuse)
            with tf.variable_scope('logits', reuse = reuse):
                return conv1d(curr_input, to_vocab_size, 'logits', reuse)
        
        self.logits = forward(self.X, target_1)
        
        masks = tf.sequence_mask(self.Y_seq_len, max_seq, dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        def cond(i, y, temp):
            return i < tf.reduce_max(max_seq)
        
        def body(i, y, temp):
            logits = forward(self.X, y, reuse = True)
            ids = tf.argmax(logits, -1)[:, i]
            ids = tf.expand_dims(ids, -1)
            temp = tf.concat([temp[:, 1:], ids], -1)
            y = tf.concat([temp[:, -(i+1):], temp[:, :-(i+1)]], -1)
            y = tf.reshape(y, [tf.shape(temp)[0], max_seq])
            i += 1
            return i, y, temp
        
        target = tf.fill([batch_size, max_seq], GO)
        target = tf.cast(target, tf.int64)
        self.target = target
        
        _, self.predicting_ids, _ = tf.while_loop(cond, body, 
                                                  [tf.constant(0), target, target])

In [11]:
residual_channels = 128
encoder_dilations = [1,2,4,8,16,1,2,4,8,16]
decoder_dilations = [1,2,4,8,16,1,2,4,8,16]
encoder_filter_width = 3
decoder_filter_width = 3
batch_size = 128
epoch = 20

In [12]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = ByteNet(len(dictionary_from), len(dictionary_to), 
                residual_channels, encoder_dilations, decoder_dilations,
                encoder_filter_width,decoder_filter_width)
sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead
bytenet_encoder_layer_0_1
Instructions for updating:
Use the `axis` argument instead
Tensor("bytenet_encoder_layer_0_1/bytenet_encoder_layer_0_1_1_1/Squeeze:0", shape=(?, ?, 128), dtype=float32)
bytenet_encoder_layer_1_2
Tensor("bytenet_encoder_layer_1_2/bytenet_encoder_layer_1_2_1_1/Squeeze:0", shape=(?, ?, 128), dtype=float32)
bytenet_encoder_layer_2_4
Tensor("bytenet_encoder_layer_2_4/bytenet_encoder_layer_2_4_1_1/Squeeze:0", shape=(?, ?, 128), dtype=float32)
bytenet_encoder_layer_3_8
Tensor("bytenet_encoder_layer_3_8/bytenet_encoder_layer_3_8_1_1/Squeeze:0", shape=(?, ?, 128), dtype=float32)
bytenet_encoder_layer_4_16
Tensor("bytenet_encoder_layer_4_16/bytenet_encoder_layer_4_16_1_1/Squeeze:0", shape=(?, ?, 128), dtype=float32)
bytenet_encoder_layer_5_1
Tensor("bytenet_encoder_layer_5_1/bytenet_encoder_layer_5_1_1_1/Squeeze:0", shape=(?, ?, 128), dtype=float32)
bytenet_encoder_layer_6_2
Tensor("bytenet_enc

In [13]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

def pad_sentence_batch(sentence_batch, sentence_batch_y, pad_int):
    x, y = [], []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    max_sentence_len_y = max([len(sentence) for sentence in sentence_batch_y])
    max_sentence_len = max(max_sentence_len, max_sentence_len_y)
    for no, sentence in enumerate(sentence_batch):
        x.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        y.append(sentence_batch_y[no] + [pad_int] * (max_sentence_len - len(sentence_batch_y[no])))
    return x, y

In [14]:
train_X = str_idx(train_X, dictionary_from)
test_X = str_idx(test_X, dictionary_from)
train_Y = str_idx(train_Y, dictionary_to)
test_Y = str_idx(test_Y, dictionary_to)

In [15]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x, batch_y = pad_sentence_batch(train_X[i : index], train_Y[i : index], PAD)
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x, batch_y = pad_sentence_batch(test_X[i : index], test_Y[i : index], PAD)
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 0/1042 [00:00<?, ?it/s]

epoch 1, training avg loss 2.536019, training avg acc 0.598162
epoch 1, testing avg loss 0.370165, testing avg acc 0.942144


minibatch loop:   0%|          | 0/1042 [00:00<?, ?it/s]

epoch 2, training avg loss 0.236962, training avg acc 0.962103
epoch 2, testing avg loss 0.197636, testing avg acc 0.972153


minibatch loop:   0%|          | 0/1042 [00:00<?, ?it/s]

...

epoch 18, training avg loss 0.016996, training avg acc 0.996231
epoch 18, testing avg loss 0.212258, testing avg acc 0.988296


minibatch loop:   0%|          | 0/1042 [00:00<?, ?it/s]

epoch 19, training avg loss 0.002376, training avg acc 0.999325
epoch 19, testing avg loss 0.227242, testing avg acc 0.988243


minibatch loop: 100%|██████████| 23/23 [00:01<00:00, 13.02it/s, accuracy=0.994, cost=0.163]

epoch 20, training avg loss 0.002386, training avg acc 0.999336
epoch 20, testing avg loss 0.224342, testing avg acc 0.987937


In [16]:
rev_dictionary_to = {int(k): v for k, v in rev_dictionary_to.items()}

In [18]:
test_size = 20

batch_x, batch_y = pad_sentence_batch(test_X[: test_size], test_Y[: test_size], PAD)
feed = {model.X: batch_x, model.Y: batch_y}
logits = sess.run(model.predicting_ids, feed_dict = feed)
logits.shape

(20, 123)

In [19]:
rejected = ['PAD', 'EOS', 'UNK', 'GO']

for i in range(test_size):
    predict = [rev_dictionary_to[i] for i in logits[i] if rev_dictionary_to[i] not in rejected]
    actual = [rev_dictionary_to[i] for i in batch_y[i] if rev_dictionary_to[i] not in rejected]
    print(i, 'predict:', ' '.join(predict))
    print(i, 'actual:', ' '.join(actual))
    print()

0 predict: oh The The The War The War The War The War The The War The War The War oh War The War and and Box The War The The War The War The War The Edward The War The War The War The oh War The War The War The War Luther War The War The War War oh oh War War The The War The War oh The War The War The War The War The War The oh Box The War The War The The War The Edward The War The War The War Edward War War The War The War War War The War The War The War The War The War oh War oh War oh War The Đất
0 actual: Làm sao tôi có thể trình bày trong <NUM> phút về sợi dây liên kết những người phụ nữ qua ba thế hệ , về việc làm thế nào những sợi dây mạnh mẽ đáng kinh ngạc ấy đã níu chặt lấy cuộc sống của một cô bé bốn tuổi co quắp với đứa em gái nhỏ của cô bé , với mẹ và bà trong suốt năm ngày đêm trên con thuyền nhỏ lênh đênh trên Biển Đông hơn <NUM> năm trước , những sợi dây liên kết đã níu lấy cuộc đời cô bé ấy và không bao giờ rời đi - - cô bé ấy giờ sống ở San Francisco và đang nói chuyện